# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

# Make it code cell when running locally
!pip install azureml-core
!pip install azureml-automl-core
!pip install azureml-widgets
!pip install azureml-train-core
!pip install azureml-train-automl-runtime
!pip install azureml-train-automl-client
!pip install azureml-train-restclients-hyperdrive
!pip install azureml-pipeline

In [62]:
import azureml.core
print(azureml.core.VERSION)

1.36.0


In [73]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive import BanditPolicy
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [74]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'mlops-capstone-project-hyperdrive-final'
project_folder = './mlops-capstone-project-folder'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
mlops-capstone-project-hyperdrive-final,MLOPS,Link to Azure Machine Learning studio,Link to Documentation


found = False
key = "Bank_Loan"
description_text = "Bank Personal Loan Dataset for Udacity capstone project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
    print('not found ... Please make sure its correct')
        


df = dataset.to_pandas_dataframe()
df.describe()

In [76]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster-demo-1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 2)

Found existing cluster, use it.
Succeeded........................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [78]:
from azureml.core import Environment
sklearn_env = Environment.from_conda_specification(name = 'scikitlearn_env', file_path = './envFile.yml')
sklearn_env

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20211029.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "scikitlearn_env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge

In [79]:
from azureml.core import ScriptRunConfig
src = ScriptRunConfig(source_directory='.',
                      compute_target=compute_target,
                      script='train.py',
                      arguments=['--C', 1.0, '--max_iter', 100],
                      environment=sklearn_env)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [80]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"--C": choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000), "--max_iter": choice(40,80,120,130,200)})

#TODO: Create your estimator and hyperdrive config : Used ScriptRun config. Estimator is deprecated
#estimator = SKLearn(source_directory='./', compute_target=compute_target, entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                         primary_metric_name="Accuracy",
                                         policy=early_termination_policy,
                                         max_total_runs=10,
                                         max_concurrent_runs=4,
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         run_config=src)

In [81]:
#TODO: Submit your experiment
hyperDrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [82]:
RunDetails(hyperDrive_run).show()
hyperDrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_6c11792a-f40c-49a1-bff4-0dea8e30265f
Web View: https://ml.azure.com/runs/HD_6c11792a-f40c-49a1-bff4-0dea8e30265f?wsid=/subscriptions/1db20fb5-44d4-452c-9f69-3e242db6cb9e/resourcegroups/ml-resource-group/workspaces/MLOPS&tid=482198bb-ae7b-4b25-8b7a-6d7f32faa083

Execution Summary
RunId: HD_6c11792a-f40c-49a1-bff4-0dea8e30265f
Web View: https://ml.azure.com/runs/HD_6c11792a-f40c-49a1-bff4-0dea8e30265f?wsid=/subscriptions/1db20fb5-44d4-452c-9f69-3e242db6cb9e/resourcegroups/ml-resource-group/workspaces/MLOPS&tid=482198bb-ae7b-4b25-8b7a-6d7f32faa083



{'runId': 'HD_6c11792a-f40c-49a1-bff4-0dea8e30265f',
 'target': 'cluster-demo-1',
 'status': 'Completed',
 'startTimeUtc': '2022-01-05T22:54:49.881928Z',
 'endTimeUtc': '2022-01-05T23:30:54.645523Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'cc8edd16-9e4e-45c5-b071-07bcb1d6fdd7',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1063-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.36.0',
  'space_size': '55',
  'score': '0.909',
  'best_child_run_id': 'HD_6c11792a-f40c-49a1-bff4-0dea8e30265f_3',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlops5538635530.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_6c11792a-f40c-49a1-bff4-0dea8e30265f/azureml-logs/hyp

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [83]:
import joblib
best_run = hyperDrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

In [84]:
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength ("--C"):',best_run_metrics['Regularization Strength:'])
print('\n Max Iterations:',best_run_metrics['Max iterations:'])
#joblib.dump(value=best_run.id, filename="./outputs/best_capstone_project_hyperdrive_model.joblib")

Best Run Id:  HD_6c11792a-f40c-49a1-bff4-0dea8e30265f_3

 Accuracy: 0.909

 Regularization Strength ("--C"): 500.0

 Max Iterations: 80


In [85]:
#TODO: Save the best model
print(os.getcwd())
os.makedirs('./hyperdrive-capstone-model',exist_ok=True)
best_run.download_file('/outputs/best_capstone_project_hyperdrive_model.joblib',os.path.join('./hyperdrive-capstone-model','best_capstone_project_hyperdrive_model.joblib'))

/mnt/batch/tasks/shared/LS_root/mounts/clusters/joy-compute-instance/code/Users/jbhattac6/azureml/capstone


In [86]:
#Register Model
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

bestModel = best_run.register_model(
                        model_path='./outputs/best_capstone_project_hyperdrive_model.joblib', 
                        model_name='HyperDrive_Capstone_Best_Model',
                        tags={'Capstone HyperDrive AML Training tag':'Capstone Hyperdrive Auto ML'},
                        model_framework=Model.Framework.SCIKITLEARN,
                        description='Logistic Regression model to predict Bank Loan.',
                        properties={'Accuracy': best_run_metrics['Accuracy']}
                        )

print(bestModel)
print('Version:', bestModel.version)

Model(workspace=Workspace.create(name='MLOPS', subscription_id='1db20fb5-44d4-452c-9f69-3e242db6cb9e', resource_group='ml-resource-group'), name=HyperDrive_Capstone_Best_Model, id=HyperDrive_Capstone_Best_Model:2, version=2, tags={'Capstone HyperDrive AML Training tag': 'Capstone Hyperdrive Auto ML'}, properties={'Accuracy': '0.909'})
Version: 2


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [66]:
#AutoML perfomed better with accuracy of 0.98. Hence skipping this step

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

